In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
from dateutil.relativedelta import relativedelta

credit = pd.read_csv("credit_record.csv").drop('Unnamed: 0', axis=1, errors='ignore')
credit['START_DATE'] = pd.to_datetime(credit['START_DATE'])
credit['SNAPSHOT_MONTH'] = pd.to_datetime(credit['SNAPSHOT_MONTH'])
credit.head()

,ID,STATUS,START_DATE,SNAPSHOT_MONTH
0,5001711,0,2022-09-01,2022-09-01
1,5001711,0,2022-09-01,2022-10-01
2,5001711,0,2022-09-01,2022-11-01
3,5001711,X,2022-09-01,2022-12-01
4,5001712,0,2021-06-01,2021-06-01


In [4]:
print("Unique values ==> ")
credit.apply(lambda x: x.nunique())

Unique values ==> 


ID                45971
STATUS                8
START_DATE           61
SNAPSHOT_MONTH       61
dtype: int64

In [6]:
print("Missing values values ==> ")
credit.apply(lambda x: x.isnull().sum())

Unique values ==> 


ID                0
STATUS            0
START_DATE        0
SNAPSHOT_MONTH    0
dtype: int64